In [ ]:
%matplotlib inline
import pysal as ps
import numpy as np
import pandas as pd
import descartes
import geopandas as gp
import shapely
import copy
import random
import os
import sys
import acs_regionalization as ACS
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 40,40  #set  the   default   map   size  
rcParams['patch.linewidth'] = 1  #set default polygon line width

In [ ]:
pwd

In [ ]:
# read in the Austin shapefile and plot it
shp_map = gp.read_file('mamba.shp')
shp_map['constant'] = 1
p = shp_map.plot('constant', alpha=0.5, cmap='gist_heat', edgecolor="k")
p.axis('off')
p.plot()

In [ ]:
# read spatial data into python and run some diagnostics
w = ps.rook_from_shapefile('mamba.shp', idVariable='GEOID')
print(w.n) # number of census tracts
print(w)
print type(w.id2i)
# # for key in w.id2i:
# #     key.encode('ascii')
# #     print (key)
# w.id2i2 = w.id2i.items    
# # w.id2i2 = [key.encode('ascii') for key in w.id2i]
# print type(w.id2i)
# # print w.id2i

# def convert(input):
#     if isinstance(input, dict):
#         return {convert(key): convert(value) for key, value in input.iteritems()}
#     elif isinstance(input, list):
#         return [convert(element) for element in input]
#     elif isinstance(input, unicode):
#         return input.encode('latin-1')
#     else:
#         return input
    
# w.id2i = convert(w.id2i)
# print id2i2

# # for key in w.id2i2:
# #     key.encode('latin-1')
# #     print(type(key))

# def RepresentsInt(i):
#     try: 
#         int(i)
#         return True
#     except ValueError:
#         return False


# for i in w.id2i:
#     print RepresentsInt(i)



In [ ]:
# summary of the connectivity structure of the Los Angeles County census tracts
w_hist_data = np.array(w.histogram)
fig, ax = plt.subplots(figsize=(15,15))
ax.bar(w_hist_data[:,0], w_hist_data[:,1], align='center', width=1)
ax.set_xlabel('number of neighbors')
ax.set_ylabel('number of tracts')

In [ ]:
#test = open('Exclude_Test.csv')
#test = test.readline()
#test = test.split(',')
#print type(test)
#test2 = [int(i) for i in test]
#print type(test2)

#def RepresentsInt(i):
 #   try: 
 #       int(i)
 #       return True
 #   except ValueError:
 #       return False


#for i in test2:
  #  print RepresentsInt(i)

In [ ]:
# read in list of census tracts to exclude
# using arcmap to view zoning data overlaid on census tracts symbolized by total population estimate per tract
exclude = open('Exclude_Test.csv')
exclude = exclude.readline()
exclude = exclude.replace('g','')
exclude = exclude.split(',')
print exclude
for i in exclude:
    i.encode('utf-8')
    print(type(i))


# # print(type(exclude))
# exclude2 = [int(i) for i in exclude]

# exclude2 = open('Exclude_Test.csv')
# exclude2 = exclude2.readline()
# exclude2 = exclude2.split(',')
# print type(exclude2)
# exclude = [int(i) for i in exclude2]
# print type(exclude)

# def RepresentsInt(i):
#     try: 
#         int(i)
#         return True
#     except ValueError:
#         return False


# for i in exclude:
#     print RepresentsInt(i)


    

In [ ]:
# read in the ACS estimates and standard errors
population = pd.read_csv('ACS_Full_Table_Copy1.csv', index_col=0)
pop2 = pd.read_csv('Total_Population.csv', index_col=0)
est_perc = pd.read_csv('prop_est_vet_perc.csv', index_col=0)
# csv file of estimated total veteran population by census tract.
est_vet_pop = pd.read_csv('proportion1.csv', index_col=0)
# standard error csv by census tract.
stderr_pop = pd.read_csv('std_err_prop.csv', index_col=0)
# margin of error calculation 
# moe_pop = stderr_pop * 1.65  # changed from 1.645 to 1.65 to reduce decimal places in analysis
moe_prop = stderr_pop * 1.654  
# read the census tract ids into a python object 
pop_tract_ids = open('tract_ids.csv')
# split the csv into a python list object
pop_tract_ids = pop_tract_ids.readline().split(',')
# calculate the estimate ratio, standard error ratio, and margin of error for each census tract as a python object
est_ratio = pd.read_csv('est_ratio5.csv', index_col=0) 
std_ratio = pd.read_csv('std_err_prop.csv', index_col=0)
# print (est_ratio.dtypes)
moe_ratio = std_ratio * 1.654

# population_ratio.csv is a CSV file with the census tract id and the above ratio names as columns.
ratio_names = open('population_ratio.csv') # need to create this file tonight
ratio_names = ratio_names.readline().split(',') 

In [ ]:
# ratio and population data is organized as numerator-denominator pairs
print ratio_names
print est_ratio.columns  # names of the numerators and denominators
print 'the numerator of %s is %s, and the denominator is %s'%(ratio_names[0], population.columns[3], population.columns[6])
print 'the numerator of %s is %s, and the denominator is %s'%(ratio_names[1], population.columns[1], population.columns[6])

#print (est_ratio.values)
#print (est_vet_pop.values)
#print (stderr_pop.values)

In [ ]:
# THIS CODE BLOCK WILL TAKE AT LEAST 19+ HOURS(SO FAR AND NOT FINISHED) TO RUN IF YOU ARE DOING ALL OF LOS ANGELES COUNTY.

# open the shapefile of the census tracts.
shp = ps.open('mamba.shp')
# using a random seed in python generates a random number and using the same seed value ensures that the same results are generated.
# if a random seed is not used, I cant reliably check that the same result comes from each execution of the code, 
# since the code uses pseudorandom numbers in the ACS Regionalization algorythm.
              
random.seed(789)     # to ensure we get the same solution each time
np.random.seed(789)  # to ensure we get the same solution each time
results = ACS.ACS_Regions(w=w,\
                          target_est_prop=est_vet_pop.values,\
                          target_moe_prop=moe_prop.values,\
                          target_est_ratio=est_ratio.values,\
                          target_moe_ratio=moe_ratio.values,\
                          count_est=population.values,\
                          target_th_all=0.05,\
                          exclude=exclude,\
                          compactness=shp,\
                          cv_exclude_prop=0.05)
print 'regionalization finished'

In [ ]:
# some basic diagnostics on the solution
print 'number of regions:', len(results.regions)
print 'total time (seconds):', results.time['total']

In [ ]:
# This is a python function to create the side by side plots of LA County

# We'll make a few side-by-side plots. This code wraps the setup into a function. This will plot out the two maps
# of LA County before and after regionalization.
def two_plot_setup():
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(40,40))
    axes[0].set_aspect('equal')
    axes[1].set_aspect('equal')
    axes[0].axis('off')
    axes[1].axis('off')
    return fig, axes

In [ ]:
#This cell calls the above function and actually creates the two plots.

fig, axes = two_plot_setup()

# color tracts based on region ID  #This shows all the 
rids = copy.copy(results.region_ids)
shp_map['rids'] = rids
shp_map.plot('rids', categorical=True, alpha=1, colormap='Dark2', axes=axes[0])

# disolve the tracts into regions
region_groups = shp_map.groupby('rids')
region_map = gp.GeoDataFrame(index=region_groups.indices.keys())
region_map['rids'] = region_groups.indices.keys()
region_map['geometry'] = region_groups.geometry.apply(shapely.ops.unary_union)
region_map.plot('rids', categorical=True, alpha=1, colormap='Dark2', axes=axes[1])